In [1]:
# dependencies
import pandas as pd

# National & State Summary - ETL

### Data to populate 5 charts comparing sociodemographic factors between US Census Tracts that are food insecure or not based on the LAhalfand10 flag: 
1. % low income households
2. % non-productive age groups (minors & seniors) by population
3. % race/ethnicity distribution by population
4. % households HUNV
5. % SNAP housing units

[documentation](https://www.ers.usda.gov/data-products/food-access-research-atlas/documentation/)


In [2]:
# Read the food access data from the S3 bucket into a DataFrame
df = pd.read_csv("https://gtbootcamp20230221.s3.amazonaws.com/FoodAccessResearchAtlasData2019.csv", dtype={'CensusTract': str})
df.head()

,CensusTract,State,County,Urban,Pop2010,OHU2010,GroupQuartersFlag,NUMGQTRS,PCTGQTRS,LILATracts_1And10,...,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP
0,01001020100,Alabama,Autauga County,1,1912,693,0,0.0,0.00,0,...,221.0,1622.0,217.0,14.0,0.0,14.0,45.0,44.0,6.0,102.0
1,01001020200,Alabama,Autauga County,1,2170,743,0,181.0,8.34,1,...,214.0,888.0,1217.0,5.0,0.0,5.0,55.0,75.0,89.0,156.0
2,01001020300,Alabama,Autauga County,1,3373,1256,0,0.0,0.00,0,...,439.0,2576.0,647.0,17.0,5.0,11.0,117.0,87.0,99.0,172.0
3,01001020400,Alabama,Autauga County,1,4386,1722,0,0.0,0.00,0,...,904.0,4086.0,193.0,18.0,4.0,11.0,74.0,85.0,21.0,98.0
4,01001020500,Alabama,Autauga County,1,10766,4082,0,181.0,1.68,0,...,1126.0,8666.0,1437.0,296.0,9.0,48.0,310.0,355.0,230.0,339.0


In [3]:
# create state FIP column for mapping
df['StateFIPS'] = df['CensusTract'].str.slice(0,2)
df.head()

,CensusTract,State,County,Urban,Pop2010,OHU2010,GroupQuartersFlag,NUMGQTRS,PCTGQTRS,LILATracts_1And10,...,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP,StateFIPS
0,01001020100,Alabama,Autauga County,1,1912,693,0,0.0,0.00,0,...,1622.0,217.0,14.0,0.0,14.0,45.0,44.0,6.0,102.0,01
1,01001020200,Alabama,Autauga County,1,2170,743,0,181.0,8.34,1,...,888.0,1217.0,5.0,0.0,5.0,55.0,75.0,89.0,156.0,01
2,01001020300,Alabama,Autauga County,1,3373,1256,0,0.0,0.00,0,...,2576.0,647.0,17.0,5.0,11.0,117.0,87.0,99.0,172.0,01
3,01001020400,Alabama,Autauga County,1,4386,1722,0,0.0,0.00,0,...,4086.0,193.0,18.0,4.0,11.0,74.0,85.0,21.0,98.0,01
4,01001020500,Alabama,Autauga County,1,10766,4082,0,181.0,1.68,0,...,8666.0,1437.0,296.0,9.0,48.0,310.0,355.0,230.0,339.0,01


In [4]:
# show names of columns
print(df.columns.tolist())

['CensusTract', 'State', 'County', 'Urban', 'Pop2010', 'OHU2010', 'GroupQuartersFlag', 'NUMGQTRS', 'PCTGQTRS', 'LILATracts_1And10', 'LILATracts_halfAnd10', 'LILATracts_1And20', 'LILATracts_Vehicle', 'HUNVFlag', 'LowIncomeTracts', 'PovertyRate', 'MedianFamilyIncome', 'LA1and10', 'LAhalfand10', 'LA1and20', 'LATracts_half', 'LATracts1', 'LATracts10', 'LATracts20', 'LATractsVehicle_20', 'LAPOP1_10', 'LAPOP05_10', 'LAPOP1_20', 'LALOWI1_10', 'LALOWI05_10', 'LALOWI1_20', 'lapophalf', 'lapophalfshare', 'lalowihalf', 'lalowihalfshare', 'lakidshalf', 'lakidshalfshare', 'laseniorshalf', 'laseniorshalfshare', 'lawhitehalf', 'lawhitehalfshare', 'lablackhalf', 'lablackhalfshare', 'laasianhalf', 'laasianhalfshare', 'lanhopihalf', 'lanhopihalfshare', 'laaianhalf', 'laaianhalfshare', 'laomultirhalf', 'laomultirhalfshare', 'lahisphalf', 'lahisphalfshare', 'lahunvhalf', 'lahunvhalfshare', 'lasnaphalf', 'lasnaphalfshare', 'lapop1', 'lapop1share', 'lalowi1', 'lalowi1share', 'lakids1', 'lakids1share', 'lase

In [5]:
# summarize total number of population or households in selected features by STATE
state_summary_df = df.groupby(['State','LAhalfand10']).agg({
    'Pop2010': 'sum',
    'OHU2010': 'sum',
    'TractLOWI': 'sum',
    'TractKids': 'sum',
    'TractSeniors': 'sum',
    'TractWhite': 'sum',
    'TractBlack': 'sum',
    'TractAsian': 'sum',
    'TractNHOPI': 'sum',
    'TractAIAN': 'sum',
    'TractOMultir': 'sum',
    'TractHispanic': 'sum'    
})
# reset index; transform State and LAhalfand10 from index fields to columns
state_summary_df.reset_index(inplace=True)

# change column name
state_summary_df.rename(columns={'State':'Jurisdiction'}, inplace=True)

# display dataframe
state_summary_df

,Jurisdiction,LAhalfand10,Pop2010,OHU2010,TractLOWI,TractKids,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic
0,Alabama,0,2011849,781527,750829.0,477968.0,285743.0,1596873.0,321048.0,13113.0,1094.0,15662.0,64059.0,66205.0
1,Alabama,1,2767887,1102264,1042096.0,654491.0,372049.0,1678521.0,930263.0,40482.0,1963.0,12556.0,104102.0,119397.0
2,Alaska,0,186563,67331,42372.0,46209.0,13900.0,136745.0,2989.0,5396.0,735.0,18974.0,14265.0,7935.0
3,Alaska,1,523668,190727,134581.0,138068.0,40634.0,336630.0,20273.0,32721.0,6674.0,78812.0,48558.0,31307.0
4,Arizona,0,857778,321996,346581.0,209192.0,117376.0,617568.0,31972.0,19973.0,2419.0,60861.0,124985.0,245099.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,West Virginia,1,921047,386789,345703.0,189943.0,150293.0,846547.0,42880.0,8959.0,289.0,2015.0,20357.0,13740.0
98,Wisconsin,0,1953902,765486,532677.0,460000.0,267095.0,1740172.0,93087.0,23524.0,561.0,23374.0,73184.0,107933.0
99,Wisconsin,1,3733084,1514282,1059719.0,879492.0,510219.0,3161895.0,266061.0,105710.0,1266.0,31152.0,167000.0,228123.0
100,Wyoming,0,137770,52261,39444.0,36212.0,15520.0,120061.0,640.0,756.0,103.0,8315.0,7895.0,13073.0


In [6]:
# summarize total number of population or households in selected features for US
us_summary_df = df.groupby(['LAhalfand10']).agg({
    'Pop2010': 'sum',
    'OHU2010': 'sum',
    'TractLOWI': 'sum',
    'TractKids': 'sum',
    'TractSeniors': 'sum',
    'TractWhite': 'sum',
    'TractBlack': 'sum',
    'TractAsian': 'sum',
    'TractNHOPI': 'sum',
    'TractAIAN': 'sum',
    'TractOMultir': 'sum',
    'TractHispanic': 'sum' 
})
# reset index; transform LAhalfand10 from index field to column
us_summary_df.reset_index(inplace=True)

# add Jurisdiction column
us_summary_df['Jurisdiction'] = 'United States'

# display data
us_summary_df


,LAhalfand10,Pop2010,OHU2010,TractLOWI,TractKids,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,Jurisdiction
0,0,91132485,34733952,31407468.0,21215438.0,12063938.0,67587011.0,10152811.0,3909156.0,129476.0,943163.0,8389823.0,14436336.0,United States
1,1,217613053,81982340,69046369.0,52957586.0,28202843.0,155965656.0,28776503.0,10765064.0,410535.0,1968964.0,19726331.0,36040953.0,United States


In [7]:
# concatenate state and US total data
summary_df = pd.concat([state_summary_df,us_summary_df])
summary_df.reset_index(inplace=True, drop=True)
summary_df.tail()

,Jurisdiction,LAhalfand10,Pop2010,OHU2010,TractLOWI,TractKids,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic
99,Wisconsin,1,3733084,1514282,1059719.0,879492.0,510219.0,3161895.0,266061.0,105710.0,1266.0,31152.0,167000.0,228123.0
100,Wyoming,0,137770,52261,39444.0,36212.0,15520.0,120061.0,640.0,756.0,103.0,8315.0,7895.0,13073.0
101,Wyoming,1,425856,174618,117110.0,99190.0,54570.0,391218.0,4108.0,3670.0,324.0,5021.0,21515.0,37158.0
102,United States,0,91132485,34733952,31407468.0,21215438.0,12063938.0,67587011.0,10152811.0,3909156.0,129476.0,943163.0,8389823.0,14436336.0
103,United States,1,217613053,81982340,69046369.0,52957586.0,28202843.0,155965656.0,28776503.0,10765064.0,410535.0,1968964.0,19726331.0,36040953.0


## Calculate percentages from totals and add to summary_df
1. % low income Population 
    - TractLOWI/Pop2010
2. % non-productive age groups (minors & seniors) by population
    - TractKids/Pop2010
    - TractSeniors/Pop2010
3. % race/ethnicity distribution by population
    - TractWhite/Pop2010
    - TractBlack/Pop2010
    - TractAsian/Pop2010
    - TractNHOPI/Pop2010
    - TractAIAN/Pop2010
    - TractOMultir/Pop2010
    - TractHispanic/Pop2010
4. % households HUNV
    - TractHUNV/OHU2010
5. % SNAP housing units
    - TractSNAP/OHU2010

# 1st Visual

In [8]:
# Low income Population
summary_df['LOWI_per'] = summary_df['TractLOWI']/summary_df['Pop2010']
summary_df = summary_df.sort_values('LOWI_per', ascending=False)

summary_df

########### not sure why some of the percentages are above 100% in the LOWI_per column.
########### is it possible the denom needs to be the population and not the household?
########### suggest someone read the data dictionary/documentation to figure this out.
############# I changed to population instead of household - Jovawnte 

,Jurisdiction,LAhalfand10,Pop2010,OHU2010,TractLOWI,TractKids,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,LOWI_per
49,Mississippi,1,1951759,735055,836101.0,498648.0,246498.0,1064463.0,808634.0,19611.0,938.0,6165.0,51948.0,58855.0,0.428383
56,Nevada,0,513082,187861,217782.0,130994.0,60235.0,327664.0,35398.0,30184.0,3027.0,6240.0,110569.0,179904.0,0.424458
7,Arkansas,1,1900847,750965,789608.0,465127.0,268048.0,1368963.0,369367.0,27240.0,4921.0,13480.0,116876.0,142552.0,0.415398
18,Florida,0,3225435,1239642,1329476.0,661500.0,564828.0,2508728.0,474426.0,45762.0,1621.0,13589.0,181309.0,835730.0,0.412185
63,New Mexico,1,1694526,646327,693011.0,432734.0,220226.0,1147476.0,37524.0,24941.0,1567.0,157358.0,325660.0,802015.0,0.408970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13,Connecticut,1,2770589,1064989,589524.0,626041.0,406291.0,2159021.0,280554.0,112369.0,1029.0,8009.0,209607.0,346393.0,0.212779
23,Hawaii,1,952246,301372,201025.0,221184.0,134526.0,231630.0,16555.0,364575.0,92751.0,2822.0,243913.0,86407.0,0.211106
59,New Hampshire,1,718473,284043,144732.0,156100.0,97091.0,663837.0,10621.0,20244.0,215.0,1724.0,21832.0,25371.0,0.201444
58,New Hampshire,0,597997,234930,116325.0,131134.0,81177.0,572213.0,4414.0,8163.0,169.0,1426.0,11612.0,11333.0,0.194524


# 2nd Visual

In [19]:
# % non-productive age groups (minors & seniors) by population
kids_seniors = summary_df['TractKids'] + summary_df['TractSeniors']

summary_df['Non_prod_ages_per'] = kids_seniors/summary_df['Pop2010']

summary_df

,Jurisdiction,LAhalfand10,Pop2010,OHU2010,TractLOWI,TractKids,TractSeniors,TractWhite,TractBlack,TractAsian,...,Non_prod_ages_per,White_per,Black_per,Asian_per,NHOPI_per,AIAN_per,Multi_Race_per,Hispanic_per,HUNV_per,SNAP_per
49,Mississippi,1,1951759,735055,836101.0,498648.0,246498.0,1064463.0,808634.0,19611.0,...,0.381782,0.545386,0.414310,0.010048,0.000481,0.003159,0.026616,0.030155,0.164087,0.243034
56,Nevada,0,513082,187861,217782.0,130994.0,60235.0,327664.0,35398.0,30184.0,...,0.372707,0.638619,0.068991,0.058829,0.005900,0.012162,0.215500,0.350634,0.025131,0.086911
7,Arkansas,1,1900847,750965,789608.0,465127.0,268048.0,1368963.0,369367.0,27240.0,...,0.385710,0.720186,0.194317,0.014330,0.002589,0.007092,0.061486,0.074994,0.063315,0.133147
18,Florida,0,3225435,1239642,1329476.0,661500.0,564828.0,2508728.0,474426.0,45762.0,...,0.380205,0.777795,0.147089,0.014188,0.000503,0.004213,0.056212,0.259106,0.043549,0.015717
63,New Mexico,1,1694526,646327,693011.0,432734.0,220226.0,1147476.0,37524.0,24941.0,...,0.385335,0.677166,0.022144,0.014719,0.000925,0.092863,0.192184,0.473298,0.013518,0.079904
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13,Connecticut,1,2770589,1064989,589524.0,626041.0,406291.0,2159021.0,280554.0,112369.0,...,0.372604,0.779264,0.101262,0.040558,0.000371,0.002891,0.075654,0.125025,0.081908,0.131413
23,Hawaii,1,952246,301372,201025.0,221184.0,134526.0,231630.0,16555.0,364575.0,...,0.373548,0.243246,0.017385,0.382858,0.097402,0.002964,0.256145,0.090740,0.073374,0.095053
59,New Hampshire,1,718473,284043,144732.0,156100.0,97091.0,663837.0,10621.0,20244.0,...,0.352402,0.923955,0.014783,0.028176,0.000299,0.002400,0.030387,0.035312,0.078779,0.144264
58,New Hampshire,0,597997,234930,116325.0,131134.0,81177.0,572213.0,4414.0,8163.0,...,0.355037,0.956883,0.007381,0.013651,0.000283,0.002385,0.019418,0.018952,0.022513,0.076253


# 3rd Visual

In [10]:
# % race/ethnicity distribution by population
# TractWhite/Pop2010
# TractBlack/Pop2010
# TractAsian/Pop2010
# TractNHOPI/Pop2010
# TractAIAN/Pop2010
# TractOMultir/Pop2010
# TractHispanic/Pop2010
summary_df['White_per'] = summary_df['TractWhite']/summary_df['Pop2010']
summary_df['Black_per'] = summary_df['TractBlack']/summary_df['Pop2010']
summary_df['Asian_per'] = summary_df['TractAsian']/summary_df['Pop2010']
summary_df['NHOPI_per'] = summary_df['TractNHOPI']/summary_df['Pop2010']
summary_df['AIAN_per'] = summary_df['TractAIAN']/summary_df['Pop2010']
summary_df['Multi_Race_per'] = summary_df['TractOMultir']/summary_df['Pop2010']
summary_df['Hispanic_per'] = summary_df['TractHispanic']/summary_df['Pop2010']

summary_df

,Jurisdiction,LAhalfand10,Pop2010,OHU2010,TractLOWI,TractKids,TractSeniors,TractWhite,TractBlack,TractAsian,...,TractHispanic,LOWI_per,Non_prod_ages_per,White_per,Black_per,Asian_per,NHOPI_per,AIAN_per,Multi_Race_per,Hispanic_per
49,Mississippi,1,1951759,735055,836101.0,498648.0,246498.0,1064463.0,808634.0,19611.0,...,58855.0,0.428383,0.381782,0.545386,0.414310,0.010048,0.000481,0.003159,0.026616,0.030155
56,Nevada,0,513082,187861,217782.0,130994.0,60235.0,327664.0,35398.0,30184.0,...,179904.0,0.424458,0.372707,0.638619,0.068991,0.058829,0.005900,0.012162,0.215500,0.350634
7,Arkansas,1,1900847,750965,789608.0,465127.0,268048.0,1368963.0,369367.0,27240.0,...,142552.0,0.415398,0.385710,0.720186,0.194317,0.014330,0.002589,0.007092,0.061486,0.074994
18,Florida,0,3225435,1239642,1329476.0,661500.0,564828.0,2508728.0,474426.0,45762.0,...,835730.0,0.412185,0.380205,0.777795,0.147089,0.014188,0.000503,0.004213,0.056212,0.259106
63,New Mexico,1,1694526,646327,693011.0,432734.0,220226.0,1147476.0,37524.0,24941.0,...,802015.0,0.408970,0.385335,0.677166,0.022144,0.014719,0.000925,0.092863,0.192184,0.473298
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13,Connecticut,1,2770589,1064989,589524.0,626041.0,406291.0,2159021.0,280554.0,112369.0,...,346393.0,0.212779,0.372604,0.779264,0.101262,0.040558,0.000371,0.002891,0.075654,0.125025
23,Hawaii,1,952246,301372,201025.0,221184.0,134526.0,231630.0,16555.0,364575.0,...,86407.0,0.211106,0.373548,0.243246,0.017385,0.382858,0.097402,0.002964,0.256145,0.090740
59,New Hampshire,1,718473,284043,144732.0,156100.0,97091.0,663837.0,10621.0,20244.0,...,25371.0,0.201444,0.352402,0.923955,0.014783,0.028176,0.000299,0.002400,0.030387,0.035312
58,New Hampshire,0,597997,234930,116325.0,131134.0,81177.0,572213.0,4414.0,8163.0,...,11333.0,0.194524,0.355037,0.956883,0.007381,0.013651,0.000283,0.002385,0.019418,0.018952


# 4th Visual

In [13]:
summary_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 104 entries, 49 to 61
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Jurisdiction       104 non-null    object 
 1   LAhalfand10        104 non-null    int64  
 2   Pop2010            104 non-null    int64  
 3   OHU2010            104 non-null    int64  
 4   TractLOWI          104 non-null    float64
 5   TractKids          104 non-null    float64
 6   TractSeniors       104 non-null    float64
 7   TractWhite         104 non-null    float64
 8   TractBlack         104 non-null    float64
 9   TractAsian         104 non-null    float64
 10  TractNHOPI         104 non-null    float64
 11  TractAIAN          104 non-null    float64
 12  TractOMultir       104 non-null    float64
 13  TractHispanic      104 non-null    float64
 14  LOWI_per           104 non-null    float64
 15  Non_prod_ages_per  104 non-null    float64
 16  White_per          104 non

In [15]:
#% households HUNV
#TractHUNV/OHU2010
summary_df['HUNV_per'] = df['TractHUNV']/df['OHU2010']

summary_df

,Jurisdiction,LAhalfand10,Pop2010,OHU2010,TractLOWI,TractKids,TractSeniors,TractWhite,TractBlack,TractAsian,...,LOWI_per,Non_prod_ages_per,White_per,Black_per,Asian_per,NHOPI_per,AIAN_per,Multi_Race_per,Hispanic_per,HUNV_per
49,Mississippi,1,1951759,735055,836101.0,498648.0,246498.0,1064463.0,808634.0,19611.0,...,0.428383,0.381782,0.545386,0.414310,0.010048,0.000481,0.003159,0.026616,0.030155,0.164087
56,Nevada,0,513082,187861,217782.0,130994.0,60235.0,327664.0,35398.0,30184.0,...,0.424458,0.372707,0.638619,0.068991,0.058829,0.005900,0.012162,0.215500,0.350634,0.025131
7,Arkansas,1,1900847,750965,789608.0,465127.0,268048.0,1368963.0,369367.0,27240.0,...,0.415398,0.385710,0.720186,0.194317,0.014330,0.002589,0.007092,0.061486,0.074994,0.063315
18,Florida,0,3225435,1239642,1329476.0,661500.0,564828.0,2508728.0,474426.0,45762.0,...,0.412185,0.380205,0.777795,0.147089,0.014188,0.000503,0.004213,0.056212,0.259106,0.043549
63,New Mexico,1,1694526,646327,693011.0,432734.0,220226.0,1147476.0,37524.0,24941.0,...,0.408970,0.385335,0.677166,0.022144,0.014719,0.000925,0.092863,0.192184,0.473298,0.013518
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13,Connecticut,1,2770589,1064989,589524.0,626041.0,406291.0,2159021.0,280554.0,112369.0,...,0.212779,0.372604,0.779264,0.101262,0.040558,0.000371,0.002891,0.075654,0.125025,0.081908
23,Hawaii,1,952246,301372,201025.0,221184.0,134526.0,231630.0,16555.0,364575.0,...,0.211106,0.373548,0.243246,0.017385,0.382858,0.097402,0.002964,0.256145,0.090740,0.073374
59,New Hampshire,1,718473,284043,144732.0,156100.0,97091.0,663837.0,10621.0,20244.0,...,0.201444,0.352402,0.923955,0.014783,0.028176,0.000299,0.002400,0.030387,0.035312,0.078779
58,New Hampshire,0,597997,234930,116325.0,131134.0,81177.0,572213.0,4414.0,8163.0,...,0.194524,0.355037,0.956883,0.007381,0.013651,0.000283,0.002385,0.019418,0.018952,0.022513


# 5th Visual

In [17]:
#% SNAP housing units
#TractSNAP/OHU2010
summary_df['SNAP_per'] = df['TractSNAP']/df['OHU2010']

summary_df


,Jurisdiction,LAhalfand10,Pop2010,OHU2010,TractLOWI,TractKids,TractSeniors,TractWhite,TractBlack,TractAsian,...,Non_prod_ages_per,White_per,Black_per,Asian_per,NHOPI_per,AIAN_per,Multi_Race_per,Hispanic_per,HUNV_per,SNAP_per
49,Mississippi,1,1951759,735055,836101.0,498648.0,246498.0,1064463.0,808634.0,19611.0,...,0.381782,0.545386,0.414310,0.010048,0.000481,0.003159,0.026616,0.030155,0.164087,0.243034
56,Nevada,0,513082,187861,217782.0,130994.0,60235.0,327664.0,35398.0,30184.0,...,0.372707,0.638619,0.068991,0.058829,0.005900,0.012162,0.215500,0.350634,0.025131,0.086911
7,Arkansas,1,1900847,750965,789608.0,465127.0,268048.0,1368963.0,369367.0,27240.0,...,0.385710,0.720186,0.194317,0.014330,0.002589,0.007092,0.061486,0.074994,0.063315,0.133147
18,Florida,0,3225435,1239642,1329476.0,661500.0,564828.0,2508728.0,474426.0,45762.0,...,0.380205,0.777795,0.147089,0.014188,0.000503,0.004213,0.056212,0.259106,0.043549,0.015717
63,New Mexico,1,1694526,646327,693011.0,432734.0,220226.0,1147476.0,37524.0,24941.0,...,0.385335,0.677166,0.022144,0.014719,0.000925,0.092863,0.192184,0.473298,0.013518,0.079904
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13,Connecticut,1,2770589,1064989,589524.0,626041.0,406291.0,2159021.0,280554.0,112369.0,...,0.372604,0.779264,0.101262,0.040558,0.000371,0.002891,0.075654,0.125025,0.081908,0.131413
23,Hawaii,1,952246,301372,201025.0,221184.0,134526.0,231630.0,16555.0,364575.0,...,0.373548,0.243246,0.017385,0.382858,0.097402,0.002964,0.256145,0.090740,0.073374,0.095053
59,New Hampshire,1,718473,284043,144732.0,156100.0,97091.0,663837.0,10621.0,20244.0,...,0.352402,0.923955,0.014783,0.028176,0.000299,0.002400,0.030387,0.035312,0.078779,0.144264
58,New Hampshire,0,597997,234930,116325.0,131134.0,81177.0,572213.0,4414.0,8163.0,...,0.355037,0.956883,0.007381,0.013651,0.000283,0.002385,0.019418,0.018952,0.022513,0.076253
